## Predict the demand for bike share using known tools

Our goal is to predict demand for bike share based on [this](https://www.kaggle.com/c/bike-sharing-demand) Kaggle task.
Kaggle provides two data sets: a labelled train data and an unlabelled test data.
We have to use the train data to predict labels for the test data.
Kaggle won't give us the labels just a score we achieved on the test set.

### Know your data

In [ ]:
import pandas as pd
import numpy as np

bike_data = pd.read_csv("https://raw.githubusercontent.com/divenyijanos/ceu-ml/2023/data/bike_sharing_demand/bike_sample.csv")

In [ ]:
bike_data.head()

In [ ]:
bike_data.describe()

In [ ]:
bike_data.shape

In [ ]:
bike_data.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt
bike_data["datetime"] = pd.to_datetime(bike_data["datetime"])
bike_2011 = bike_data[bike_data["datetime"].dt.year == 2011]
daily_counts = bike_2011.groupby(bike_2011["datetime"].dt.date)["count"].sum()
dates = daily_counts.index
counts = daily_counts.values

plt.bar(dates, counts, color="crimson")
plt.xlabel("Date")
plt.ylabel("Rental Count")
plt.title("Daily Rentals for 2011")
plt.xticks(rotation=45)
plt.show()

### Train-test split

In [ ]:
# train-validation split on numeric features
from sklearn.model_selection import train_test_split

# keep numeric features
features = bike_data.drop(columns=["count"]).select_dtypes(include=np.number)
label = bike_data["count"]
np.random.seed(20240306)
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2)

### Technical detour: best practice for pseudo random number generation and ensuring reproducibility

We used to ensure reproducibility by setting the global seed with `np.random.seed()`.

This is a risky practice as it modifies the global state. Some imported packages and functions may be unintentionally affected.

The recommended practice instead is to create a new (psuedo) random number generator and pass it around.
Call `np.random.RandomState(<seed>)` to create a new RNG.
(For numpy 1.17+, there is a statistically better alternative: `np.random.default_rng(<seed>)`. However, not all packages are already updated to accept that class as a random state.)

See more details [here](https://scikit-learn.org/stable/common_pitfalls.html#controlling-randomness).

In [ ]:
prng = np.random.RandomState(20240306)
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=prng)

### Evaluation function

In [ ]:
# define loss function
def calculateRMSLE(prediction, y_obs):
    # TODO

### Benchmark

In [ ]:
# estimate benchmark model
benchmark = # TODO
benchmark_result = ["Benchmark", calculateRMSLE(benchmark, y_train), calculateRMSLE(benchmark, y_train)]

In [ ]:
# collect results into a DataFrame
result_columns = ["Model", "Train", "Test"]
pd.DataFrame([benchmark_result], columns=result_columns)

### Model #1: Group averages

In [ ]:
# model #1: group averages by season, holiday and workingday
from sklearn.linear_model import LinearRegression

features1 = ["season", "holiday", "workingday"]
feature_matrix1 = pd.get_dummies(X_train[features1], columns=features1, drop_first=True)
feature_matrix1

In [ ]:
# evaluate model #1
def calculateRMSLE(prediction, observation):
    return np.mean((prediction - observation)**2)

group_avg = LinearRegression().fit(feature_matrix1, y_train)
train_error = calculateRMSLE(group_avg.predict(feature_matrix1), y_train)

# for test error we need to apply the same encoding on the test set
test_error = calculateRMSLE(group_avg.predict(pd.get_dummies(X_test[features1], columns=features1, drop_first=True)), y_test)
group_avg_result = ["Group-avg", train_error, test_error]
group_avg_result

In [ ]:
pd.DataFrame([benchmark_result, group_avg_result], columns=result_columns)

### Technical detour: Using Pipelines from scikit-learn

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

one_hot_encoder = OneHotEncoder(sparse_output=False, drop="first")
steps = [
    ("dummify_selected_columns", ColumnTransformer([("dummify", one_hot_encoder, features1)])),
    ("ols", LinearRegression())
]
pipe_group_avg = Pipeline(steps)
pipe_group_avg

In [ ]:
pipe_group_avg.fit(X_train, y_train)

In [ ]:
pd.DataFrame({
    "feature": pipe_group_avg["dummify_selected_columns"].get_feature_names_out(),
    "coefficient": pipe_group_avg["ols"].coef_
})

In [ ]:
# Double-check we got the same result
[
    calculateRMSLE(pipe_group_avg.predict(X_train), y_train),
    calculateRMSLE(pipe_group_avg.predict(X_test), y_test)
]

### Model #2: Group averages with weather

In [ ]:
# Model #2: Group averages with weather
dummy_features = ['season', 'holiday', 'workingday', 'weather']
numeric_features = ['temp', 'atemp', 'humidity', 'windspeed']

steps = [
    ("dummify_selected_columns", ColumnTransformer([
        ("dummify", one_hot_encoder, dummy_features),
        ("keep", "passthrough", numeric_features)
    ])),
    ("ols", LinearRegression())
]

In [ ]:
# TODO: build a pipeline from the steps, fit the model, and evaluate

### Model #3: Very flexible linear with polynomial features

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

steps = [
    ("dummify_selected_columns", ColumnTransformer([
        ("dummify", one_hot_encoder, dummy_features),
        ("keep", "passthrough", numeric_features)
    ])),
    ("4_degree_poly", PolynomialFeatures(degree=4, include_bias=False)),
    ("ols", LinearRegression())
]
pipe_flexible_linear = Pipeline(steps)
pipe_flexible_linear

In [ ]:
pipe_flexible_linear.fit(X_train, y_train)
train_error = calculateRMSLE(pipe_flexible_linear.predict(X_train), y_train)
test_error = calculateRMSLE(pipe_flexible_linear.predict(X_test), y_test)

flexible_linear_result = ['Flexible linear', train_error, test_error]
flexible_linear_result
results.loc[len(results)] = flexible_linear_result
results

### Model #4: Improve with Lasso

In [ ]:
# Model #4: improve with Lasso
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import MinMaxScaler

steps = [
    ("dummify_selected_columns", ColumnTransformer([
        ("dummify", one_hot_encoder, dummy_features),
        ("scale", MinMaxScaler(), numeric_features)
    ])),
    ("4_degree_poly", PolynomialFeatures(degree=4, include_bias=False)),
    # TODO: include the estimation step, choose the best penalty parameter via CV
]
pipe_lasso = Pipeline(steps)

pipe_lasso.fit(X_train, y_train)

In [ ]:
train_error = calculateRMSLE(pipe_lasso.predict(X_train), y_train)
test_error = calculateRMSLE(pipe_lasso.predict(X_test), y_test)

lasso_model_result = ['Flexible LASSO', train_error, test_error]
results.loc[len(results)] = lasso_model_result
results

### Model #5: Regression tree

In [ ]:
from sklearn import tree

steps = [
    ("tree", tree.DecisionTreeRegressor(max_depth=5))
]
pipe_tree = Pipeline(steps)
pipe_tree

In [ ]:
pipe_tree.fit(X_train, y_train)

train_error = calculateRMSLE(pipe_tree.predict(X_train), y_train)
test_error = calculateRMSLE(pipe_tree.predict(X_test), y_test)

tree_result = ['Tree', train_error, test_error]
results.loc[len(results)] = tree_result
results

In [ ]:
plt.figure(figsize=(12,12))
tree.plot_tree(pipe_tree["tree"],feature_names=dummy_features + numeric_features)

## Improve the models

### Diagnostics

In [ ]:
import matplotlib.pyplot as plt

linear_predictions = pipe_group_avg_with_weather.predict(X_test)
lasso_predictions = pipe_lasso.predict(X_test)
tree_predictions = pipe_tree.predict(X_test)

plt.scatter(y_test, linear_predictions, label='Linear', alpha=0.5)
plt.scatter(y_test, lasso_predictions, label='Lasso', alpha=0.5)
plt.scatter(y_test, tree_predictions, label='Tree', alpha=0.5)
plt.axline((1, 1), slope=1, linestyle='dashed', color='red')
plt.xlabel('Observed')
plt.ylabel('Predicted')
plt.legend()

In [ ]:
bike_data[bike_data['count'] < 10]

### Feature engineering

In [ ]:
def extract_dt_features(df_with_datetime):
    df_with_datetime['datetime'] = pd.to_datetime(df_with_datetime['datetime'], utc=True)
    df_with_datetime['year'] = df_with_datetime['datetime'].dt.year
    df_with_datetime['day'] = df_with_datetime['datetime'].dt.day
    df_with_datetime['month'] = df_with_datetime['datetime'].dt.month
    df_with_datetime['hour'] = df_with_datetime['datetime'].dt.hour
    df_with_datetime['dayofweek'] = df_with_datetime['datetime'].dt.dayofweek


extract_dt_features(bike_data)

In [ ]:
feature_matrix = bike_data.drop(columns=["count", "registered", "casual"]).select_dtypes(include=np.number)
label = bike_data["count"]
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, label, test_size=0.2, random_state=20240306)

In [ ]:
dummy_features = ['season', 'holiday', 'workingday', 'weather', 'year', 'month', 'day', 'hour', 'dayofweek']

steps = [
    ("dummify_selected_columns", ColumnTransformer([
        ("dummify", one_hot_encoder, dummy_features),
        ("keep", "passthrough", numeric_features)
    ])),
    ("ols", LinearRegression())
]
pipe_linear = Pipeline(steps)
pipe_linear.fit(X_train, y_train)

In [ ]:
train_error = calculateRMSLE(pipe_linear.predict(X_train), y_train)
test_error = calculateRMSLE(pipe_linear.predict(X_test), y_test)

linear_FE_result = ['Feature engineered linear', train_error, test_error]
results.loc[len(results)] = linear_FE_result
results

In [ ]:
# TODO: LASSO


In [ ]:
# TODO: Tree
tree_fe = tree.DecisionTreeRegressor(max_depth=5).fit(X_train, y_train)

train_error = calculateRMSLE(tree_fe.predict(X_train), y_train)
test_error = calculateRMSLE(tree_fe.predict(X_test), y_test)

tree_fe_result = ['Feature engineered tree', train_error, test_error]
results.loc[len(results)] = tree_fe_result
results

### Collect more data

In [ ]:
bike_full = pd.read_csv("https://raw.githubusercontent.com/divenyijanos/ceu-ml/2023/data/bike_sharing_demand/train.csv")
bike_full.shape

In [ ]:
bike_data.shape

In [ ]:
extract_dt_features(bike_full)

In [ ]:
bike_full.shape

In [ ]:
# Ensure the test set remains intact
full_data_without_original_test = bike_full.loc[~bike_full.datetime.isin(bike_data.filter(X_test.index, axis=0)['datetime'])]
full_data_without_original_test.shape

In [ ]:
X_full = full_data_without_original_test.drop(columns=["count", "registered", "casual", "datetime"])
y_full = full_data_without_original_test['count']

In [ ]:
# Linear

pipe_linear.fit(X_full, y_full)
train_error = calculateRMSLE(pipe_linear.predict(X_full), y_full)
test_error = calculateRMSLE(pipe_linear.predict(X_test), y_test)

linear_FE_full_result = ['Feature engineered linear large n', train_error, test_error]
results.loc[len(results)] = linear_FE_full_result
results

In [ ]:
# Lasso
# TODO

In [ ]:
# Tree
# TODO

### Apply more flexible models

In [ ]:
# live coding

## Submit to Kaggle

In [ ]:
# live coding